# Technique for Order of Preference by Similarity to Ideal Solution (AHP-TOPSIS)

##### Modified from: https://www.kaggle.com/code/hungrybluedev/topsis-implementation/notebook </br>

It is a multi-criteria decision analysis method that is based on the concept that the chosen alternative should have the shortest geometric distance to the Positive Ideal Solution (PIS) and the longest geometric solution from the Negative Ideal Solution (NIS).

# AHP Weight calculation
1.  Pair-wise comparison of each criteria and sub-criteria to establish the weight of the supply chain parameters.
2. Global summation of all these weights (weighted arithmetic sum) for each alternative and ordering them on the basis of this weighted sum.
3. Calculate the consistency ratio which should be less than 0.10, otherwise the weights are not balanced

## AHP Pair-wise Comparison Matrix
As the first step a pairwise comparison matrix is determined qualitatively according to the priority using the Saaty 9 point scale (Evelyn, E. et.al, 2015). This table is subjectively evaluated, as AHP combines both quantitative and qualitative aspects in the decision method, which helps the analysis to find the best possible answer rather than a correct solution (Longaray, A.A. et.al, 2015).
<br /><br />

| Features              | Digital Prepardness  | Natural Disaster | Labor Strikes | Political Instability | Logistic Index |
|-----------------------|----------------------|------------------|---------------|-----------------------|----------------|
| Digital Prepardness   | 1                    | 6                | 8             | 4                     | 3              |
| Natural Disaster      | 1/6                  | 1                | 3             | 1/2                   | 1/4            |
| Labor Strikes         | 1/8                  | 1/3              | 1             | 1/2                   | 1/4            |
| Political Instability | 1/4                  | 2                | 2             | 1                     | 1/3            |
| Logistic Index        | 1/3                  | 5                | 4             | 3                     | 1              |

In [13]:
from ahpy import ahpy

supply_chain_comp = {
    ('Digital Prepardness', 'Digital Prepardness'): 1, ('Digital Prepardness', 'Natural Disasters'): 6, ('Digital Prepardness', 'Labor Strikes'): 8, ('Digital Prepardness', 'Political Instability'): 4, ('Digital Prepardness', 'Logistic Index'): 3,
    ('Natural Disasters', 'Digital Prepardness'): 1/6, ('Natural Disasters', 'Natural Disasters'): 1, ('Natural Disasters', 'Labor Strikes'): 3, ('Natural Disasters', 'Political Instability'): 1/2, ('Natural Disasters', 'Logistic Index'): 1/5,
    ('Labor Strikes', 'Digital Prepardness'): 1/8, ('Labor Strikes', 'Natural Disasters'): 1/3, ('Labor Strikes', 'Labor Strikes'): 1, ('Labor Strikes', 'Political Instability'): 1/2, ('Labor Strikes', 'Logistic Index'): 1/4,
    ('Political Instability', 'Digital Prepardness'): 1/4, ('Political Instability', 'Natural Disasters'): 2, ('Political Instability', 'Labor Strikes'): 2, ('Political Instability', 'Political Instability'): 1, ('Political Instability', 'Logistic Index'): 1/3,
    ('Logistic Index', 'Digital Prepardness'): 1/3, ('Logistic Index', 'Natural Disasters'): 5, ('Logistic Index', 'Labor Strikes'): 4, ('Logistic Index', 'Political Instability'): 3, ('Logistic Index', 'Logistic Index'): 1
    }


## Calculate the weight for the criteria
The above table represents an n x n comparison matrix which contains the intensities defined by us (Karmaker, C.L et.al, 2018). $$
 M = (w_{i}/w_{j})_{n x n} = (\begin{array}{cc}
w_{1}/w_{1} & w_{1}/w_{2} & .... & w_{1}/w_{n}\\
w_{2}/w_{1} & w_{2}/w_{2} & .... & w_{2}/w_{n}\\
. & . & ..... & . \\
w_{n}/w_{1} & w_{n}/w_{2} & .... & w_{n}/w_{n}
\end{array})
$$

## Generate the weights
The n x n matrix is first normalized using the formula below and a priority vector is generated with which the weights are generated.
$$
W = \begin{bmatrix}
           w_{1} \\
           w_{2} \\
           \vdots \\
           w_{n}
         \end{bmatrix}
$$

In [14]:
supply_chain = ahpy.Compare(name='Supply_Chain', comparisons=supply_chain_comp, precision=3, random_index='saaty')

print(supply_chain.target_weights)
print('Consistency Ratio: ' + str(supply_chain.consistency_ratio))

{'Digital Prepardness': 0.498, 'Logistic Index': 0.26, 'Political Instability': 0.111, 'Natural Disasters': 0.08, 'Labor Strikes': 0.05}
Consistency Ratio: 0.051


## Calculate and check the Consistency ratio
It is assumed that the experts using the AHP makers are objective. However, this is not true in real life which generates a certain level of uncertainty towards this evaluation. Therefore, Saaty solved this issue by creating the consistency Index and consistency ratios. To accept the matrix, the CR value should be less the 0.1 meaning there could be an inconsistency error of 10%.

In [15]:
# All the packages that we need to import
import numpy as np               # for linear algebra
import pandas as pd              # for tabular output
from scipy.stats import rankdata # for ranking the candidates

## Pre-requisites

For this problem, we are always provided with the following data:
1. The ratings in every category for each candidate.
2. The weights for every category or attribute to be considered.

Note that an attribute can be beneficial attribute (in which case, we will want to maximize it's contribution) or a cost attribute (which we will need to minimize). We call the set of beneficial attributes $J_1$ and that of cost attributes $J_2 = J_1^C$.

In [16]:
# The given data encoded into vectors and matrices

attributes = np.array(["Digital Prepardness", "Natural Disasters", "Labor Strikes", "Political Instability", "Logistic Index"])
candidates = np.array(["Germany", "Poland", "France", "Belgium", "Bulgaria", "Romania"])
raw_data = np.array([
    [690, 3.1,  9,  7,  4],
    [590, 3.9,  7,  6, 10],
    [600, 3.6,  8,  8,  7],
    [620, 3.8,  7, 10,  6],
    [700, 2.8, 10,  4,  6],
    [650, 4.0,  6,  9,  8],
])

weights = np.array([0.3, 0.2, 0.2, 0.15, 0.15])

# The indices of the attributes (zero-based) that are considered beneficial.
# Cost benefit functions 0 when cost (lower is better) and 1 when benefit function (more is better)
benefit_attributes = set([0, 1, 2, 3, 4])

# Display the raw data we have
pd.DataFrame(data=raw_data, index=candidates, columns=attributes)

,Digital Prepardness,Natural Disasters,Labor Strikes,Political Instability,Logistic Index
Germany,690.0,3.1,9.0,7.0,4.0
Poland,590.0,3.9,7.0,6.0,10.0
France,600.0,3.6,8.0,8.0,7.0
Belgium,620.0,3.8,7.0,10.0,6.0
Bulgaria,700.0,2.8,10.0,4.0,6.0
Romania,650.0,4.0,6.0,9.0,8.0


## Step 1 - Normalizing the ratings

$$r_{ij}=\frac{x_{ij}}{\sqrt{\sum_{i = 1}^{m} x_{ij}^2}}$$

where $i = 1, 2, \ldots, m$ and $j = 1, 2, \ldots, n$.

In [17]:
m = len(raw_data)
n = len(attributes)
divisors = np.empty(n)
for j in range(n):
    column = raw_data[:,j]
    divisors[j] = np.sqrt(column @ column)

raw_data /= divisors

columns = ["$X_{%d}$" % j for j in range(n)]
pd.DataFrame(data=raw_data, index=candidates, columns=columns)

,$X_{0}$,$X_{1}$,$X_{2}$,$X_{3}$,$X_{4}$
Germany,0.438053,0.355454,0.462299,0.376322,0.230556
Poland,0.374567,0.447184,0.359566,0.322562,0.576390
France,0.380916,0.412785,0.410932,0.430083,0.403473
Belgium,0.393613,0.435718,0.359566,0.537603,0.345834
Bulgaria,0.444402,0.321055,0.513665,0.215041,0.345834
Romania,0.412659,0.458650,0.308199,0.483843,0.461112


## Step 2 - Calculating the Weighted Normalized Ratings

$$v_{ij} = w_j r_{ij}$$

where $i = 1, 2, \ldots, m$ and $j = 1, 2, \ldots, n$.

In [18]:
raw_data *= weights
pd.DataFrame(data=raw_data, index=candidates, columns=columns)

,$X_{0}$,$X_{1}$,$X_{2}$,$X_{3}$,$X_{4}$
Germany,0.131416,0.071091,0.092460,0.056448,0.034583
Poland,0.112370,0.089437,0.071913,0.048384,0.086459
France,0.114275,0.082557,0.082186,0.064512,0.060521
Belgium,0.118084,0.087144,0.071913,0.080640,0.051875
Bulgaria,0.133321,0.064211,0.102733,0.032256,0.051875
Romania,0.123798,0.091730,0.061640,0.072576,0.069167


## Step 3 - Identifying PIS ($A^*$) and NIS ($A^-$)

$$
\begin{align}
A^* &= \left\{v_1^*, v_2^*, \ldots, v_n^*\right\} \\
A^- &= \left\{v_1^-, v_2^-, \ldots, v_n^-\right\} \\
\end{align}
$$

And we define

$$
\begin{align}
v_j^* &=
\begin{cases}
\max{(v_{ij})}, \text{ if} j \in J_1 \\
\min{(v_{ij})}, \text{ if} j \in J_2
\end{cases}
\\
v_j^- &=
\begin{cases}
\min{(v_{ij})}, \text{ if} j \in J_1 \\
\max{(v_{ij})}, \text{ if} j \in J_2
\end{cases}
\\
\end{align}
$$

where $i = 1, 2, \ldots, m$ and $j = 1, 2, \ldots, n$.

In [19]:
a_pos = np.zeros(n)
a_neg = np.zeros(n)
for j in range(n):
    column = raw_data[:,j]
    max_val = np.max(column)
    min_val = np.min(column)
    
    # See if we want to maximize benefit or minimize cost (for PIS)
    if j in benefit_attributes:
        a_pos[j] = max_val
        a_neg[j] = min_val
    else:
        a_pos[j] = min_val
        a_neg[j] = max_val

pd.DataFrame(data=[a_pos, a_neg], index=["$A^*$", "$A^-$"], columns=columns)

,$X_{0}$,$X_{1}$,$X_{2}$,$X_{3}$,$X_{4}$
$A^*$,0.133321,0.091730,0.102733,0.080640,0.086459
$A^-$,0.112370,0.064211,0.061640,0.032256,0.034583


## Step 4 and 5 - Calculating Separation Measures and Similarities to PIS

The separation or distance between the alternatives can be measured by the $n$-dimensional Euclidean distance. The separation from the PIS $A^*$ and NIS $A^-$ are $S^*$ and $S^-$ respectively.

$$
\begin{align}
S_i^* &= \sqrt{\sum_{j = 1}^n \left(v_{ij} - v^*_j\right)^2} \\
S_i^- &= \sqrt{\sum_{j = 1}^n \left(v_{ij} - v^-_j\right)^2} \\
\end{align}
$$

where $i = 1, 2, \ldots, m$ and $j = 1, 2, \ldots, n$.

We also calculate

$$
C^*_i = \frac{S_i^-}{S_i^* + S_i^-},\text{ where }i = 1, 2, \ldots, m
$$

In [20]:
sp = np.zeros(m)
sn = np.zeros(m)
cs = np.zeros(m)

for i in range(m):
    diff_pos = raw_data[i] - a_pos
    diff_neg = raw_data[i] - a_neg
    sp[i] = np.sqrt(diff_pos @ diff_pos)
    sn[i] = np.sqrt(diff_neg @ diff_neg)
    cs[i] = sn[i] / (sp[i] + sn[i])

pd.DataFrame(data=zip(sp, sn, cs), index=candidates, columns=["$S^*$", "$S^-$", "$C^*$"])

,$S^*$,$S^-$,$C^*$
Germany,0.061737,0.044104,0.416704
Poland,0.049341,0.060770,0.551900
France,0.042449,0.049755,0.539620
Belgium,0.048980,0.057482,0.539926
Bulgaria,0.065531,0.049260,0.429128
Romania,0.046297,0.060907,0.568142


## Step 6 - Ranking the candidates/alternatives

We choose the candidate with the maximum $C^*$ or rank all the alternatives in descending order according to their $C^*$ values. This process can also be done for the $S^*$ and $S^-$ values.

In [21]:
def rank_according_to(data):
    ranks = rankdata(data).astype(int)
    ranks -= 1
    return candidates[ranks][::-1]

In [22]:
cs_order = rank_according_to(cs)
sp_order = rank_according_to(sp)
sn_order = rank_according_to(sn)

pd.DataFrame(data=zip(cs_order, sp_order, sn_order), index=range(1, m + 1), columns=["$C^*$", "$S^*$", "$S^-$"])

,$C^*$,$S^*$,$S^-$
1,Romania,Poland,Romania
2,Poland,Romania,Poland
3,Belgium,France,Belgium
4,France,Germany,France
5,Bulgaria,Belgium,Bulgaria
6,Germany,Bulgaria,Germany


In [23]:
print("The best candidate/alternative according to C* is " + cs_order[0])
print("The preferences in descending order are " + ", ".join(cs_order) + ".")

The best candidate/alternative according to C* is Romania
The preferences in descending order are Romania, Poland, Belgium, France, Bulgaria, Germany.


## References
Evelyn, E. and EdmondYeboah, N., (2015). Ranking Agricultural Supply Chain Risk in Ghana: An AHP Approach. International Journal of Economics,Commerece and Management, III, 2, pp.1-12.

Hungrybluedev (2021) Topsis implementation, Kaggle. Available from: https://www.kaggle.com/code/hungrybluedev/topsis-implementation/notebook (Accessed: 12 May 2023).

Karmaker, C.L., Ahmed, S.M.T., Rahman, M.S., Tahiduzzaman, M., Biswas, T.K., Rahman, M. and Biswas, S.K., 2018. A framework of faculty performance evaluation: A case study in Bangladesh. International Journal of Research in Advanced Engineering and Technology, 4(3), pp.18-24.

Longaray, A.A., Gois, J.D.D.R. and da Silva Munhoz, P.R., ()2015. Proposal for using AHP method to evaluate the quality of services provided by outsourced companies. Procedia Computer Science, 55, pp.715-724.